In [ ]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [2]:
import os
import cv2
# print(os.path.isfile('/gdrive/My Drive/hackathon/model.h5'))

In [3]:
import argparse
import sys
import time
# from keras.applications.inception_v3 import InceptionV3
from keras.applications.nasnet import NASNetLarge
from  keras.models import load_model
import cv2
import numpy as np


nas = NASNetLarge(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
# nas.summary()
nas.layers.pop()
nas.outputs = [nas.layers[-1].output]
nas.layers[-1].outbound_nodes = []

Using TensorFlow backend.
W0914 18:00:11.152097 4465030592 deprecation_wrapper.py:119] From /Users/aldermann/anaconda3/envs/Harassment/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0914 18:00:11.205059 4465030592 deprecation_wrapper.py:119] From /Users/aldermann/anaconda3/envs/Harassment/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 18:00:11.208562 4465030592 deprecation_wrapper.py:119] From /Users/aldermann/anaconda3/envs/Harassment/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0914 18:00:11.249511 4465030592 deprecation_wrapper.py:119] From /Users/aldermann/anaconda3/envs/Harassment/lib/python3.6/site-packages/keras/backend/tensorflow_back

 61169664/359748576 [====>.........................] - ETA: 1:08

KeyboardInterrupt: 

In [ ]:
classifier = load_model('model.h5')

In [1]:
def add_predict(frame, predict):
    action_mapping = {0: 'Abnormal', 1: 'Normal'}
    color = (0, 255, 0)
    if predict == 0:
        color = (0, 0, 255)
    height, width, _ = frame.shape
    cv2.rectangle(frame, (0, 0), (width, height), color, 2)
    cv2.putText(frame, action_mapping[predict], (10, 15), cv2.FONT_HERSHEY_SIMPLEX, 1, color, thickness=2, lineType=2)

In [ ]:
frame_interval = 25  # Number of frames after which to run face detection
fps_display_interval = 5  # frames
frame_rate = 0
frame_count = 0

cap = cv2.VideoCapture('video.mp4')
_, frame = cap.read()
start_time = time.time()

segment = []

height, width, layers = frame.shape
fourcc = cv2.VideoWriter_fourcc(*'MPEG')
output_video = cv2.VideoWriter('output.avi', fourcc, 20.0,
                               (width, height))
predict = None
ret = True
frame_count = 0
while ret:
    ret, frame = cap.read()
    if not ret:
        continue
    try:
        image = cv2.resize(frame, (331, 331), interpolation=cv2.INTER_AREA)
        segment.append(image)
        frame_count += 1
    except Exception as e:
        continue

    if len(segment) == frame_interval:
        features = nas.predict(np.array(segment))
        result = classifier.predict(np.array(features).reshape(1, frame_interval, 4032))
        print(result)
        del segment[:]
        predict = np.argmax(result[0], axis=0)
    if predict is not None and frame_count%frame_interval<6:
        add_predict(frame, predict)
    output_video.write(frame)

# When everything is done, release the capture
cap.release()
output_video.release()
cv2.destroyAllWindows()